# Escape Game

Authors : Yana RAGOZINA - Thomas PAUL

## Imports

In [8]:
import numpy as np
import random

## Introduction

In this exercise we seek to implement an intelligent agent for an escape game task.

The agent must learn to find the most optimal way to reach room 5 starting from room 2, in a 6-room building connected by doors.

## Q-learning


The rooms in the buiding can be represented as states. The agent can move from one room to another following the doors available in each room.
Therefore, the agent can have 6 actions (= 6 possible rooms). Every movement between the rooms is followed by a reward, 100 being attributed to the final state, room 5. Other movements gain 0 reward, and -1 is assigned to impossible transitions (no door available between state n and n+1).

In [9]:
# Constants
states = 6
actions = 6
gamma = 0.8
episodes = 100

R = np.array([
    [ -1, -1, -1, -1, 0, -1 ],
    [ -1, -1, -1, 0, -1, 100 ],
    [ -1, -1, -1, 0, -1, -1 ],
    [ -1, 0, 0, -1, 0, -1 ],
    [ 0, -1, -1, 0, -1, 100 ],
    [ -1, 0, -1, -1, 0, 100 ],
])

We initialize the Q-table as follows: 

(-1 is assigned to impossible transitions)

In [10]:
# 1. Initialize the Q-table
Q = np.array([
    [ -1, -1, -1, -1, 0, -1 ], # state (room) 0
    [ -1, -1, -1, 0, -1, 0 ],  # state 1
    [ -1, -1, -1, 0, -1, -1 ], # state 2
    [ -1, 0, 0, -1, 0, -1 ],   # state 3
    [ 0, -1, -1, 0, -1, 0 ],   # state 4
    [ -1, 0, -1, -1, 0, 0 ],   # state 5
])


In [11]:
# 2.a Using Bellman equation, compute the new value of Q(1,5)
# 2.b Update the Q-table accordingly
Q[1, 5] = R[1, 5] + gamma * np.max(Q[5,:])

print(Q)

[[ -1  -1  -1  -1   0  -1]
 [ -1  -1  -1   0  -1 100]
 [ -1  -1  -1   0  -1  -1]
 [ -1   0   0  -1   0  -1]
 [  0  -1  -1   0  -1   0]
 [ -1   0  -1  -1   0   0]]


The agent receives the maximum reward, 100, reaching the final state

In [12]:
# 3. For the next episode, we start with a randomly chosen initial state. 
# This time, we have state 3 as our initial state and by random selection,  
# we select to go to state 1 as our action. Update the Q(3,1) value
Q[3, 1] = R[3, 1] + gamma * np.max(Q[1,:])

print(f"Q[3, 1]\n {Q}")

#The next state, 1, now becomes the current state.  We repeat the inner loop of the Q 
#learning algorithm because state 1 is not the goal state. Assume we select again randomly 
#state 5. What is the new updated Q-table at the end of this episode ?
Q[1, 5] = R[1, 5] + gamma * np.max(Q[5,:])

print(f"Q[1, 5]\n {Q}")


Q[3, 1]
 [[ -1  -1  -1  -1   0  -1]
 [ -1  -1  -1   0  -1 100]
 [ -1  -1  -1   0  -1  -1]
 [ -1  80   0  -1   0  -1]
 [  0  -1  -1   0  -1   0]
 [ -1   0  -1  -1   0   0]]
Q[1, 5]
 [[ -1  -1  -1  -1   0  -1]
 [ -1  -1  -1   0  -1 100]
 [ -1  -1  -1   0  -1  -1]
 [ -1  80   0  -1   0  -1]
 [  0  -1  -1   0  -1   0]
 [ -1   0  -1  -1   0   0]]


In [13]:
# 4. Try to explore more episodes to find the Q-table reaching the convergence values.
for episode in range(episodes):

    state = random.randint(0, 5)

    while state != 5:

        target_action = random.randint(0, 5)

        while R[state, target_action] == -1:
            target_action = random.randint(0, 5)

        Q[state, target_action] = R[state, target_action] + gamma * np.max(Q[target_action,:])

        state = target_action


print(Q)

[[ -1  -1  -1  -1  80  -1]
 [ -1  -1  -1  64  -1 100]
 [ -1  -1  -1  64  -1  -1]
 [ -1  80  51  -1  80  -1]
 [ 64  -1  -1  64  -1 100]
 [ -1   0  -1  -1   0   0]]


The Q-table starts to converge, highligting the most and the least probable transitions from state n to state n+1.

It is normal that the Q[5, 5] is equal to 0 because we exit the loop once we have reached this state

We can also observe the probabilities of the agent's movements in each state. For example, in state 3 the agent will be most likely to move either to room 1 or room 4 (having equal probabilities) and will be the least likely to move to state 2

In [15]:
# Once the matrix Q gets close enough to a state of convergence, we know our agent has 
# learned the most optimal paths to the goal state.  Tracing the best sequences of states is as 
# simple as following the links with the highest values at each state. 
# What is the best sequence to escape the building from room 2 ?

state = 2
steps = [2]

while state != 5:
    next_step_index = np.argmax(Q[state, :])
    steps.append(next_step_index)
    state = next_step_index

print(steps)

[2, 3, 1, 5]


The agent has followed one of optimal paths according to the given schema of the building.